In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from torchvision import transforms, models
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

In [ ]:
# import libraries
import albumentations
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F

import torchvision

from sklearn import metrics, model_selection, preprocessing

In [ ]:
from pathlib import Path

path_input = Path('/kaggle/input/happy-whale-and-dolphin')
def path_exploration():
    for path in path_input.iterdir():
        print(path)    
#path_exploration()
file_sample = "sample_submission.csv"
folders = ['train_images', 'test_images']
df = pd.read_csv(path_input/'train.csv')
df.head()

In [ ]:
# path and datas
#root_dir = '/kaggle/input/plant-pathology-2021-fgvc8/'
root_dir = '/kaggle/input/happy-whale-and-dolphin/'
#IMAGE_PATH = root_dir + 'train_images/'
IMAGE_PATH = root_dir + 'train_images/'
#IMAGE_PATH_TEST = root_dir + 'img_sz_256/'
IMAGE_PATH_TEST  = root_dir + 'test_images/'
IMAGE_SIZE = (64,96)
current_folder  = "/kaggle/working/"
LR = 3e-3
EPOCHS=6
TRAIN_BS=16
VALID_BS=16
N_classes = 30

In [ ]:
# read the data
def get_data(root_dir_csv):
    df_raw = pd.read_csv(root_dir_csv  + 'train.csv')
    df_raw.rename(columns={'species': 'labels'}, inplace=True)
    print(df_raw.shape)
    print(df_raw.head())

    # label encoding
    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(df_raw['labels'])
    df_raw['labels'] = label_encoder.transform(df_raw['labels'])
    #print(df_raw.head())
    #df = df_raw.drop(['labels'], axis = 1).reset_index(drop=True)
 
    print(df.head())
    df_raw.to_csv(current_folder+'df.csv', index=False)
    return label_encoder, df_raw

label_encoder, df_raw = get_data(root_dir)
df = pd.read_csv(current_folder + 'df.csv')
print(df.head())
print(df.shape)

def make_train_valid_df():
    df_train, df_valid = model_selection.train_test_split(
        df,
        test_size=0.2,
        random_state=42,
        stratify=df.labels.values
    )
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    print(df_train.shape)
    print(df_valid.shape)
    print(df_train.head())

    df_train.to_csv(current_folder +'df_train.csv', index=False)
    df_valid.to_csv(current_folder +'df_valid.csv', index=False)

make_train_valid_df()

df_train = pd.read_csv(current_folder + 'df_train.csv')
df_valid = pd.read_csv(current_folder + 'df_valid.csv')

# define toy_dataset


In [ ]:
df = pd.read_csv(root_dir  + 'train.csv')
list_species = sorted(df.species.unique())
print(list_species)

In [ ]:
dict_mapping = {}
for item, idx in enumerate(list_species):
    dict_mapping[item] = idx
print(dict_mapping)

int_to_name = {}
for k,v in dict_mapping.items():
    int_to_name[v] = k
print(int_to_name)

In [ ]:
dict_mapping[9]

In [ ]:
# define the dataset
def datasets():
    # define the dataset
    train_image_paths = [os.path.join(IMAGE_PATH,x) for x in df_train.image.values]
    train_targets = [x for x in df_train.labels.values]

    valid_image_paths = [os.path.join(IMAGE_PATH,x) for x in df_valid.image.values]
    valid_targets = [x for x in df_valid.labels.values]


    #test_image_paths = [f.as_posix() for f in (Path(IMAGE_PATH_TEST)).iterdir()]
    #test_targets = df_sample.labels.values

    assert(len(train_image_paths) == len(train_targets))
    assert(len(valid_image_paths) == len(valid_targets))
    return train_image_paths, train_targets, valid_image_paths, valid_targets
#assert(len(test_image_paths) == len(test_targets))
train_image_paths, train_targets, valid_image_paths, valid_targets = datasets()

In [ ]:
print(f'traindataset:{len(train_targets)}, validdataset:{len(valid_targets)}')
print(f'traindataset:{len(train_image_paths)}, validdataset:{len(valid_image_paths)}')

In [ ]:
# create the dataset

train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std = [0.229, 0.224, 0.225]),
    ])




class MyDataset(Dataset):
    def __init__(self, image_paths, targets,  transform=None):
        self.image_paths = image_paths[0:10000]
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        #image = read_image(self.image_paths[item])
        image = Image.open(self.image_paths[item]).convert('RGB')
        label = self.targets[item]
        if self.transform:
            image = self.transform(image)
        return image, label



train_dataset = MyDataset(train_image_paths, train_targets, transform = train_transform)
valid_dataset = MyDataset(valid_image_paths, valid_targets, transform = train_transform)
print(len(train_dataset))
x, y = train_dataset[1]
print(x.size())
print(type(x))
print(x[0:4, 0:4])
print(y)
# to do: save the picture in there dimensions.

In [ ]:
# model 
model_resnet18 = models.resnet18(pretrained=True)
num_ftrs = model_resnet18.fc.in_features
print(num_ftrs)
model_resnet18.fc = nn.Linear(num_ftrs, N_classes)

In [ ]:
# logistic regression
# Create Logistic Regression Model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        # Linear part
        self.linear = nn.Linear(input_dim, output_dim)
        # There should be logistic function right?
        # However logistic function in pytorch is in loss function
        # So actually we do not forget to put it, it is only at next parts
    
    def forward(self, x):
        out = self.linear(x)
        return out

# Instantiate Model Class
input_dim = 3*IMAGE_SIZE[0]*IMAGE_SIZE[1] # size of image px*px
output_dim = N_classes # labels 0,1,2,3,4,5,6,7,8,9

# create logistic regression model
model = LogisticRegressionModel(input_dim, output_dim)

# Cross Entropy Loss  
loss_func = nn.CrossEntropyLoss()

# SGD Optimizer 
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)
#in_image = (torch.rand((4,3, 32,32))).to(device)
#in_image.size()
model.to(device)
#output = model(in_image)
#outputs, loss, metrics = output
#print(output.size())

In [ ]:
#print(model_resnet18)
mean=[0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
train_dl = DataLoader(dataset=train_dataset, batch_size=TRAIN_BS, shuffle=True)
valid_dl = DataLoader(dataset=valid_dataset, batch_size=VALID_BS, shuffle=False)
x, y = next(iter(train_dl))
print(x.size())
print(y.size())


In [ ]:
# loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet18.parameters(), lr = LR)

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)
in_image = (torch.rand((4,3, 32,32))).to(device)
in_image.size()
model_resnet18.to(device)
output = model_resnet18(in_image)
#outputs, loss, metrics = output
print(output.size())

In [ ]:
outputs

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)
in_image = (torch.rand((4,3, 32,32))).to(device)
in_image.size()
model.to(device)
output = model(in_image)
#outputs, loss, metrics = output
print(output.size())

In [ ]:
val, ind = outputs.max(dim=1)
print(val)
print(ind)
print(loss)
print(metrics)

In [ ]:
model_resnet18.to(device)
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

# define learning rate scheduler
#opt = setoptim(cnn_model)
#lr_scheduler = ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=20, verbose=1)
#for i in range(100):
#    lr_scheduler.step(1)

# metrics
def metrics_batch(output, target):
    """count the number of correct predictions per data batch"""
    # get output class
    pred = output.argmax(dim=1, keepdim=True)
    # compare output class with target class
    corrects = pred.eq(target.view_as(pred)).sum().item()
    # equivlent to: correts = (pred==target).sum()
    return corrects

# compute the loss per batch
def loss_batch(loss_func, output, target, opt=None):
    """compute the loss per batch"""
    loss = loss_func(output, target)
    with torch.no_grad():
        metric_b = metrics_batch(output,target)
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
    return loss.item(), metric_b

# compute the loss value and the metric for the entire dataset (epoch)
def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    """compute the loss and metric for the entire dataset"""
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        # move batch to device
        xb = xb.to(device)
        yb = yb.to(device)
        # get model output
        output = model(xb)
        # get loss per batch
        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)
        # update running loss
        running_loss += loss_b
        # update running metric
        if metric_b is not None:
            running_metric += metric_b

        # break the loop in case of sanity check:
        if sanity_check is True:
            break
    # average loss value
    loss = running_loss/float(len_data)
    # average metric value
    metric = running_metric/float(len_data)
    return loss, metric

def train_val(model, params):
    """main function for trainning and evaluating the model on the dataset"""
    # extract model parameters
    num_epochs = params["num_epochs"]
    loss_func = params["loss_func"]
    opt = params["optimizer"]
    train_dl = params["train_dl"]
    val_dl = params["val_dl"]
    sanity_check = params["sanity_check"]
    lr_scheduler = params["lr_scheduler"]
    path2weights = params["path2weights"]

    # history of loss values in each epoch
    loss_history = {"train": [], "val": []}
    # history of metric values in each epoch
    metric_history = {"train": [], "val": []}

    # make a deep copy of weights for the best performing model
    best_model_wts = copy.deepcopy(model.state_dict())

    # initialize the best loss to a large value (infinite)
    best_loss = float("inf")

    # define a loop to calculate the loss over an epoch
    for epoch in range(num_epochs):
        # get current learning rate
        current_lr = get_lr(opt)
        print(f'Epoch {epoch}/{num_epochs-1}, current lr={current_lr}')
        # train model on training dataset
        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        # collect loss and metric for training dataset
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)

        # evaluate mode on the validation dataset
        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl, sanity_check)

            # collect loss and metric for the validation dataset
            loss_history["val"].append(val_loss)
            metric_history['val'].append(val_metric)

            # store best model
            if val_loss < best_loss:
                best_loss = val_loss
                best_model_wts  = copy.deepcopy(model.state_dict())
                # store weights into a local file
                torch.save(model.state_dict(), path2weights)
                print("copied best model weights")

            # update the learning rate schedule
            lr_scheduler.step(val_loss)
            if current_lr != get_lr(opt):
                print("loading best model weights!")
                model.load_state_dict(best_model_wts)

            # print the loss and accuracy values and return the trained model
            print(f'train loss: {train_loss:.6f}, dev loss: {val_loss:.6f}, accuracy:{100*val_metric}')
            print("-"*10)
            #load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_history, metric_history


# dict to define model parameters
params_model = {"input_shape":(3,64,128),
                "initial_filters": 8,
                "num_fc1": 100,
                "dropout_rate": 0.25,
                "num_classes": 30}



# loss function, optimizer, learning_rate
# we can either use nn.NLLLoss with the logsoftmax for the output activation of the network
# or we can use the CrossEntropyLoss with the output activation as they are (logits)
# loss_func = nn.NLLLoss(reduction='sum')
#loss_func = nn.CrossEntropyLoss(reduction='sum')

#opt = optim.Adam(model_resnet18.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(optimizer,mode='min', factor=0.5, patience=20, verbose=1)

# params_train
params_train = {"num_epochs": 10,
                "optimizer": optimizer,
                "loss_func": loss_func,
                "train_dl": train_dl,
                "val_dl": valid_dl,
                "sanity_check": False,
                "lr_scheduler": lr_scheduler,
                "path2weights": "/kaggle/working/weights.pt"}

# train and validate the model
#cnn_model, loss_hist, metric_hist = train_val(model_resnet18, params_train)
cnn_model, loss_hist, metric_hist = train_val(model_resnet18, params_train)

# Train - validation progess
num_epochs = params_train["num_epochs"]

# plot loss progress
plt.subplot(211)
plt.title("Train-Val Loss")
plt.plot(range(1, num_epochs + 1), loss_hist["train"], label="train")
plt.plot(range(1, num_epochs + 1), loss_hist["val"], label="val")
plt.ylabel("Loss")
plt.xlabel("Training epochs")
plt.legend()
plt.show()

# plot accuracy progress
plt.subplot(212)
#plt.title("Train-Val Accuracy")
plt.plot(range(1, num_epochs + 1), metric_hist["train"], label="train")
plt.plot(range(1, num_epochs + 1), metric_hist["val"], label="val")
plt.ylabel("Accuracy")
plt.xlabel("Training epochs")
plt.legend()

plt.show()

In [ ]:
6144/64


In [ ]:
path_data = Path('/kaggle/working')
for f in path_data.iterdir():
    print(f)

In [ ]:
# inference
def re_normalize(x, mean = mean, std = std):
        x_r = x.clone()
        for c, (mean_c, std_c) in enumerate(zip(mean,std)):
            x_r[c] *= std_c
            x_r[c] += mean_c
        return x_r

    # display figure
def plot_fig(img):
    plt.figure(figsize=(20, 20))
    img_r = re_normalize(img)
    ax1 = plt.subplot(1, 4, 1)
    ax1.set_title("image")
    plt.imshow(to_pil_image(img_r))

In [ ]:
def inference(n):
        #model = Net(8)
        model = model_resnet18
        model.load_state_dict(torch.load("/kaggle/working/weights.pt"))
        model.eval()
        model = model.to("cpu")


        dataiter = next(iter(valid_dl))
        images, targets = dataiter[0], dataiter[1]
        img1 = images[n]
        target = targets[n]
        output = model(img1.unsqueeze(0))
        pred = torch.argmax(output.squeeze(), dim=0).detach().cpu().numpy()
        plot_fig(img1)
        return img1, target,pred

In [ ]:
img1,target, pred = inference(25)
classe = pred.item()
#print(int_to_name[pred])
print(f'target:{target}, pred:{pred}, specy pred:{dict_mapping[classe]}')
print(f'specy name:{dict_mapping[target.item()]}')

In [ ]:
dataiter = next(iter(valid_dl))
print(len(dataiter))